In [ ]:
import numpy as np
import pandas as pd
import sklearn.cluster
import yellowbrick.cluster
import unittest
import os
import sys
import warnings
warnings.filterwarnings("ignore")

task_path = os.path.abspath(os.path.join(os.getcwd(),".."))
sys.path.append(task_path)
from src.task4 import *
utils_path = os.path.abspath(os.path.join(os.getcwd(),"..","tests"))
sys.path.append(utils_path)
from utils import *

In [ ]:
# Load all input and output datasets. These will be used in the validation step
train_features = pd.read_csv(os.path.join(os.getcwd(),"..","task4","wine_train_features.csv"), index_col=0)
#print('train_features : \n',train_features)
test_features  = pd.read_csv(os.path.join(os.getcwd(),"..","task4","wine_test_features.csv"), index_col=0)
#print('test_features : \n',test_features)
train_targets  = pd.read_csv(os.path.join(os.getcwd(),"..","task4","wine_train_targets.csv"), index_col=0)
#print('train_targets : \n',train_targets)
test_targets   = pd.read_csv(os.path.join(os.getcwd(),"..","task4","wine_test_targets.csv"), index_col=0)
#print('test_targets : \n',test_targets)
logreg_importance_df_ans = pd.read_pickle(os.path.join(os.getcwd(),"..","task4","pkl_files","wine_logreg_importance.pkl"))
#print('logreg_importance_df_ans : \n',importance_df_ans)
rf_importance_df_ans = pd.read_pickle(os.path.join(os.getcwd(),"..","task4","pkl_files","wine_rf_importance.pkl"))
#print('rf_importance_df_ans : \n',importance_df_ans)
gb_importance_df_ans = pd.read_pickle(os.path.join(os.getcwd(),"..","task4","pkl_files","wine_gb_importance.pkl"))
#print('gb_importance_df_ans : \n',importance_df_ans)

## `calculate_naive_metrics`

A Naive model is a very simple model/prediction that can help to frame how well any more sophisticated model is doing. Since the Naive Model is incredibly basic (often a constant result or a randomly selected result), we should expect that any more sophisticated model that we train should outperform it. If the Naive Model beats a trained model, it can mean that additional data (rows or columns) is needed in the dataset to improve the model or that the dataset doesn't have a strong enough signal for the target we want to predict. 
In this function you will use the approach of a constant output Naive model. You will calculate 4 metrics (`accuracy`,`recall`,`precision` and `fscore`) for the training and test datasets for a given constant integer as your prediction (passed into the function as the variable `naive_assumption`). 


##### Useful Resources
* https://machinelearningmastery.com/how-to-develop-and-evaluate-naive-classifier-strategies-using-probability/
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score

##### INPUTS
* `train_features` - a dataset split by a function similar to the tts function you created in task2
* `test_features` - a dataset split by a function similar to the tts function you created in task2
* `train_targets` - a dataset split by a function similar to the tts function you created in task2
* `test_targets` - a dataset split by a function similar to the tts function you created in task2
* `naive_assumption` - an integer that should be used as the result from the Naive model

##### OUTPUTS 
A completed `ModelMetrics` object with a training and test metrics dictionary with each one of the metrics **rounded to 4 decimal places**

In [ ]:
# Write your code here
def calculate_naive_metrics(train_features:pd.DataFrame, test_features:pd.DataFrame, train_targets:pd.Series, test_targets:pd.Series, naive_assumption:int) -> ModelMetrics:
    # TODO: Write the necessary code to calculate accuracy, recall, precision and fscore given a train and test dataframe
    # and a train and test target series and naive assumption 

    train_metrics = {
        "accuracy" : 0,
        "recall" : 0,
        "precision" : 0,
        "fscore" : 0
        }
    test_metrics = {
        "accuracy" : 0,
        "recall" : 0,
        "precision" : 0,
        "fscore" : 0
        }
    
    naive_metrics = ModelMetrics("Naive",train_metrics,test_metrics,None)
    return naive_metrics

In [ ]:
# Run this cell to test your code
naive_assumption = 1
# Answers for Wine Dataset
import pickle
with open(os.path.join(os.getcwd(),"..","task4","pkl_files","naive_metrics.pkl"), 'rb') as file:
    naive_metrics_ans = pickle.load(file)
train_accuracy_ans = naive_metrics_ans.train_metrics["accuracy"]
train_recall_ans = naive_metrics_ans.train_metrics["recall"]
train_precision_ans = naive_metrics_ans.train_metrics["precision"]
train_fscore_ans = naive_metrics_ans.train_metrics["fscore"]
test_accuracy_ans = naive_metrics_ans.test_metrics["accuracy"]
test_recall_ans = naive_metrics_ans.test_metrics["recall"]
test_precision_ans = naive_metrics_ans.test_metrics["precision"]
test_fscore_ans = naive_metrics_ans.test_metrics["fscore"]
# Calculate Metrics with Student's Function
naive_metrics = calculate_naive_metrics(train_features, test_features, train_targets, test_targets, naive_assumption)

train_accuracy = naive_metrics.train_metrics["accuracy"]
assert train_accuracy == train_accuracy_ans
train_recall = naive_metrics.train_metrics["recall"]
assert train_recall == train_recall_ans
train_precision = naive_metrics.train_metrics["precision"]
assert train_precision == train_precision_ans
train_fscore = naive_metrics.train_metrics["fscore"]
assert train_fscore == train_fscore_ans
test_accuracy = naive_metrics.test_metrics["accuracy"]
assert test_accuracy == test_accuracy_ans
train_recall = naive_metrics.test_metrics["recall"]
assert train_recall == test_recall_ans
train_precision = naive_metrics.test_metrics["precision"]
assert train_precision == test_precision_ans
train_fscore = naive_metrics.test_metrics["fscore"]
assert train_fscore == test_fscore_ans

print('Passed...')

## `calculate_logistic_regression_metrics`

A Logistic Regression model is a simple and more explainable statistical model that can be used to estimate the probability of an event (log-odds). At a high level, a Logistic Regression model uses data in the training set to estimate a column's weight in a linear approximation function (conceptually this is similar to estimating `m` for each column in the line formula you probably know well: `y = m*x + b`). If you are interested in learning more, you can read up on the Math behind how this works. For this project, we are more focused on showing you how to apply these models, so you can just use the sklearn Logistic Regression model in your code. 

For this task use Sklearn's Logistic Regression class and complete the following subtasks:
* Train a Logistic Regression model (initialized using the kwargs passed into the function)
* Predict scores for training and test datasets and calculate the 7 metrics listed below for the training and test datasets using predictions from the fit model. (All rounded to 4 decimal places)
  * `accuracy`
  * `recall`
  * `precision`
  * `fscore`
  * `false positive rate (fpr)`
  * `false negative rate (fnr)`
  * `Area Under the Curve of Receiver Operating Characteristics Curve (roc_auc)`
* Use RFE to select the top 10 features 
* Train a model using these selected features (initialized using the kwargs passed into the function)
* Create a Feature Importance DataFrame from the model trained on the top 10 features: 
  * Use the top 10 features sort by absolute value of the coefficient from biggest to smallest 
  * Make sure you use the same feature and importance column names as ModelMetrics shows in feat_name_col [`Feature`] and imp_col [`Importance`] 
  * round the importances to 4 decimal places
  * Reset the index to 0-9 you can do this the same way you did in task1 

**NOTE:** Make sure you use the predicted probabilities for roc auc

##### Useful Resources
* https://stats.libretexts.org/Bookshelves/Introductory_Statistics/OpenIntro_Statistics_(Diez_et_al)./08%3A_Multiple_and_Logistic_Regression/8.04%3A_Introduction_to_Logistic_Regression
* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
* https://en.wikipedia.org/wiki/Confusion_matrix
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html

##### INPUTS
* `train_features` - a dataset split by a function similar to the tts function you created in task2
* `test_features` - a dataset split by a function similar to the tts function you created in task2
* `train_targets` - a dataset split by a function similar to the tts function you created in task2
* `test_targets` - a dataset split by a function similar to the tts function you created in task2
* `logreg_kwargs` - a dictionary with keyword arguments that can be passed directly to the sklearn Logistic Regression class

##### OUTPUTS 
* A completed `ModelMetrics` object with a training and test metrics dictionary with each one of the metrics **rounded to 4 decimal places**
* An sklearn Logistic Regression model object fit on the training set

In [ ]:
# Write your code here
def calculate_logistic_regression_metrics(train_features:pd.DataFrame, test_features:pd.DataFrame, train_targets:pd.Series, test_targets:pd.Series, logreg_kwargs) -> tuple[ModelMetrics,LogisticRegression]:
    # TODO: Write the necessary code to train a logistic regression binary classifiaction model and calculate accuracy, recall, precision, fscore, 
    # false positive rate, false negative rate and area under the reciever operator curve given a train and test dataframe and train and test target series 
    # and keyword arguments for the logistic regrssion model

    model = LogisticRegression()
    train_metrics = {
        "accuracy" : 0,
        "recall" : 0,
        "precision" : 0,
        "fscore" : 0,
        "fpr" : 0,
        "fnr" : 0,
        "roc_auc" : 0
        }
    test_metrics = {
        "accuracy" : 0,
        "recall" : 0,
        "precision" : 0,
        "fscore" : 0,
        "fpr" : 0,
        "fnr" : 0,
        "roc_auc" : 0
        }

    log_reg_importance = pd.DataFrame()
    log_reg_metrics = ModelMetrics("Logistic Regression",train_metrics,test_metrics,log_reg_importance)

    return log_reg_metrics,model

In [ ]:
# Run this cell to test your code
logreg_kwargs = {'penalty':'l1','fit_intercept':False,'solver':'liblinear','random_state':0}
# Answers for Wine Dataset
with open(os.path.join(os.getcwd(),"..","task4","pkl_files","logreg_metrics.pkl"), 'rb') as file:
    logreg_metrics_ans = pickle.load(file)
train_accuracy_ans = logreg_metrics_ans.train_metrics["accuracy"]
train_recall_ans = logreg_metrics_ans.train_metrics["recall"]
train_precision_ans = logreg_metrics_ans.train_metrics["precision"]
train_fscore_ans = logreg_metrics_ans.train_metrics["fscore"]
train_fpr_ans = logreg_metrics_ans.train_metrics["fpr"]
train_fnr_ans = logreg_metrics_ans.train_metrics["fnr"]
train_roc_auc_ans = logreg_metrics_ans.train_metrics["roc_auc"]

test_accuracy_ans = logreg_metrics_ans.test_metrics["accuracy"]
test_recall_ans = logreg_metrics_ans.test_metrics["recall"]
test_precision_ans = logreg_metrics_ans.test_metrics["precision"]
test_fscore_ans = logreg_metrics_ans.test_metrics["fscore"]
test_fpr_ans = logreg_metrics_ans.test_metrics["fpr"]
test_fnr_ans = logreg_metrics_ans.test_metrics["fnr"]
test_roc_auc_ans = logreg_metrics_ans.test_metrics["roc_auc"]

# Calculate Metrics with Student's Function
logreg_metrics,_ = calculate_logistic_regression_metrics(train_features, test_features, train_targets, test_targets, logreg_kwargs)

train_accuracy = logreg_metrics.train_metrics["accuracy"]
assert train_accuracy == train_accuracy_ans
train_recall = logreg_metrics.train_metrics["recall"]
assert train_recall == train_recall_ans
train_precision = logreg_metrics.train_metrics["precision"]
assert train_precision == train_precision_ans
train_fscore = logreg_metrics.train_metrics["fscore"]
assert train_fscore == train_fscore_ans
train_fpr = logreg_metrics.train_metrics["fpr"]
assert train_fpr == train_fpr_ans
train_fnr = logreg_metrics.train_metrics["fnr"]
assert train_fnr == train_fnr_ans
train_roc_auc = logreg_metrics.train_metrics["roc_auc"]
assert train_roc_auc == train_roc_auc_ans
test_accuracy = logreg_metrics.test_metrics["accuracy"]
assert test_accuracy == test_accuracy_ans
test_recall = logreg_metrics.test_metrics["recall"]
assert test_recall == test_recall_ans
test_precision = logreg_metrics.test_metrics["precision"]
assert test_precision == test_precision_ans
test_fscore = logreg_metrics.test_metrics["fscore"]
assert test_fscore == test_fscore_ans
test_fpr = logreg_metrics.test_metrics["fpr"]
assert test_fpr == test_fpr_ans
test_fnr = logreg_metrics.test_metrics["fnr"]
assert test_fnr == test_fnr_ans
test_roc_auc = logreg_metrics.test_metrics["roc_auc"]
assert test_roc_auc == test_roc_auc_ans
importance_df = logreg_metrics.feat_imp_df
if compare_submission_to_answer_df(importance_df.round(4),logreg_importance_df_ans,"LR Feature Importance DF"):
    print('Passed...')
else:
    print('Failed...')

## `calculate_random_forest_metrics`

A Random Forest model is a more complex model than the Naive and Logistic Regression Models you have trained so far. It can still be used to estimate the probability of an event, but achieves this using a different underlying structure: a Tree Based Model. Conceptually, this looks a lot like lots of if/else statements chained together into a "tree". A Random Forest Expands on this and trains many different trees with different subsets of the data and starting conditions to get a better estimate than a single tree would give. For this project, we are more focused on showing you how to apply these models, so you can just use the sklearn Random Forest model in your code. 

For this task use Sklearn's Random Forest Classifier class and complete the following subtasks:
* Train a Random Forest model (initialized using the kwargs passed into the function)
* Predict scores for training and test datasets and calculate the 7 metrics listed below for the training and test datasets using predictions from the fit model. (All rounded to 4 decimal places)
  * `accuracy`
  * `recall`
  * `precision`
  * `fscore`
  * `false positive rate (fpr)`
  * `false negative rate (fnr)`
  * `Area Under the Curve of Receiver Operating Characteristics Curve (roc_auc)`
* Create a Feature Importance DataFrame from the trained model: 
  * **Do Not Use RFE for feature selection**
  * Use the top 10 features selected by the built in method (sorted from biggest to smallest) 
  * Make sure you use the same feature and importance column names as ModelMetrics shows in feat_name_col [`Feature`] and imp_col [`Importance`] 
  * Round the importances to 4 decimal places
  * Reset the index to 0-9 you can do this the same way you did in task1 

**NOTE:** Make sure you use the predicted probabilities for roc auc

##### Useful Resources
* https://blog.dataiku.com/tree-based-models-how-they-work-in-plain-english
* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
* https://en.wikipedia.org/wiki/Confusion_matrix
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html

##### INPUTS
* `train_features` - a dataset split by a function similar to the tts function you created in task2
* `test_features` - a dataset split by a function similar to the tts function you created in task2
* `train_targets` - a dataset split by a function similar to the tts function you created in task2
* `test_targets` - a dataset split by a function similar to the tts function you created in task2
* `rf_kwargs` - a dictionary with keyword arguments that can be passed directly to the sklearn RandomForestClassifier class

##### OUTPUTS 
* A completed `ModelMetrics` object with a training and test metrics dictionary with each one of the metrics **rounded to 4 decimal places**
* An sklearn Random Forest model object fit on the training set

In [ ]:
# Write your code here

def calculate_random_forest_metrics(train_features:pd.DataFrame, test_features:pd.DataFrame, train_targets:pd.Series, test_targets:pd.Series, rf_kwargs) -> tuple[ModelMetrics,RandomForestClassifier]:
    # TODO: Read the function description in https://github.gatech.edu/pages/cs6035-tools/cs6035-tools.github.io/Projects/Machine_Learning/Task4.html and implement the function as described
    
    model = RandomForestClassifier()
    train_metrics = {
        "accuracy" : 0,
        "recall" : 0,
        "precision" : 0,
        "fscore" : 0,
        "fpr" : 0,
        "fnr" : 0,
        "roc_auc" : 0
        }
    test_metrics = {
        "accuracy" : 0,
        "recall" : 0,
        "precision" : 0,
        "fscore" : 0,
        "fpr" : 0,
        "fnr" : 0,
        "roc_auc" : 0
        }

    rf_importance = pd.DataFrame()
    rf_metrics = ModelMetrics("Random Forest",train_metrics,test_metrics,rf_importance)

    return rf_metrics,model

In [ ]:
# Run this cell to test your code
rf_kwargs = {'n_estimators':150,'max_depth':3,'criterion':'log_loss','random_state':0}
# Answers for Wine Dataset
with open(os.path.join(os.getcwd(),"..","task4","pkl_files","rf_metrics.pkl"), 'rb') as file:
    rf_metrics_ans = pickle.load(file)
train_accuracy_ans = rf_metrics_ans.train_metrics["accuracy"]
train_recall_ans = rf_metrics_ans.train_metrics["recall"]
train_precision_ans = rf_metrics_ans.train_metrics["precision"]
train_fscore_ans = rf_metrics_ans.train_metrics["fscore"]
train_fpr_ans = rf_metrics_ans.train_metrics["fpr"]
train_fnr_ans = rf_metrics_ans.train_metrics["fnr"]
train_roc_auc_ans = rf_metrics_ans.train_metrics["roc_auc"]

test_accuracy_ans = rf_metrics_ans.test_metrics["accuracy"]
test_recall_ans = rf_metrics_ans.test_metrics["recall"]
test_precision_ans = rf_metrics_ans.test_metrics["precision"]
test_fscore_ans = rf_metrics_ans.test_metrics["fscore"]
test_fpr_ans = rf_metrics_ans.test_metrics["fpr"]
test_fnr_ans = rf_metrics_ans.test_metrics["fnr"]
test_roc_auc_ans = rf_metrics_ans.test_metrics["roc_auc"]

# Calculate Metrics with Student's Function
rf_metrics,_ = calculate_random_forest_metrics(train_features, test_features, train_targets, test_targets, rf_kwargs)
train_accuracy = rf_metrics.train_metrics["accuracy"]
assert train_accuracy == train_accuracy_ans
train_recall = rf_metrics.train_metrics["recall"]
assert train_recall == train_recall_ans
train_precision = rf_metrics.train_metrics["precision"]
assert train_precision == train_precision_ans
train_fscore = rf_metrics.train_metrics["fscore"]
assert train_fscore == train_fscore_ans
train_fpr = rf_metrics.train_metrics["fpr"]
assert train_fpr == train_fpr_ans
train_fnr = rf_metrics.train_metrics["fnr"]
assert train_fnr == train_fnr_ans
train_roc_auc = rf_metrics.train_metrics["roc_auc"]
assert train_roc_auc == train_roc_auc_ans
test_accuracy = rf_metrics.test_metrics["accuracy"]
assert test_accuracy == test_accuracy_ans
test_recall = rf_metrics.test_metrics["recall"]
assert test_recall == test_recall_ans
test_precision = rf_metrics.test_metrics["precision"]
assert test_precision == test_precision_ans
test_fscore = rf_metrics.test_metrics["fscore"]
assert test_fscore == test_fscore_ans
test_fpr = rf_metrics.test_metrics["fpr"]
assert test_fpr == test_fpr_ans
test_fnr = rf_metrics.test_metrics["fnr"]
assert test_fnr == test_fnr_ans
test_roc_auc = rf_metrics.test_metrics["roc_auc"]
assert test_roc_auc == test_roc_auc_ans
importance_df = rf_metrics.feat_imp_df
if compare_submission_to_answer_df(importance_df.round(4),rf_importance_df_ans,"RF Feature Importance DF"):
    print('Passed...')
else:
    print('Failed...')

## `calculate_gradient_boosting_metrics`

A Gradient Boosted model is a more complex model than the Naive and Logistic Regression Models and similar in structure to the Random Forest Model you just trained. A Gradient Boosted Model Expands on the Tree Based Model by using its additional trees to predict the errors from the previous tree. For this project, we are more focused on showing you how to apply these models, so you can just use the sklearn Gradient Boosted Model in your code. 

For this task use Sklearn's Gradient Boosting Classifier class and complete the following subtasks:
* Train a Gradient Boosted model (initialized using the kwargs passed into the function)
* Predict scores for training and test datasets and calculate the 7 metrics listed below for the training and test datasets using predictions from the fit model. (All rounded to 4 decimal places)
  * `accuracy`
  * `recall`
  * `precision`
  * `fscore`
  * `false positive rate (fpr)`
  * `false negative rate (fnr)`
  * `Area Under the Curve of Receiver Operating Characteristics Curve (roc_auc)`
* Create a Feature Importance DataFrame from the trained model: 
  * **Do Not Use RFE for feature selection**
  * Use the top 10 features selected by the built in method (sorted from biggest to smallest) 
  * Make sure you use the same feature and importance column names as ModelMetrics shows in feat_name_col [`Feature`] and imp_col [`Importance`]
  * round the importances to 4 decimal places 
  * Reset the index to 0-9 you can do this the same way you did in task1 

**NOTE:** Make sure you use the predicted probabilities for roc auc

##### Useful Resources
* https://blog.dataiku.com/tree-based-models-how-they-work-in-plain-english
* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
* https://en.wikipedia.org/wiki/Confusion_matrix
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html

##### INPUTS
* `train_features` - a dataset split by a function similar to the tts function you created in task2
* `test_features` - a dataset split by a function similar to the tts function you created in task2
* `train_targets` - a dataset split by a function similar to the tts function you created in task2
* `test_targets` - a dataset split by a function similar to the tts function you created in task2
* `gb_kwargs` - a dictionary with keyword arguments that can be passed directly to the sklearn GradientBoostingClassifier class

##### OUTPUTS 
* A completed `ModelMetrics` object with a training and test metrics dictionary with each one of the metrics **rounded to 4 decimal places**
* An sklearn Gradient Boosted model object fit on the training set

In [ ]:
# Write your code here
def calculate_gradient_boosting_metrics(train_features:pd.DataFrame, test_features:pd.DataFrame, train_targets:pd.Series, test_targets:pd.Series, gb_kwargs) -> tuple[ModelMetrics,GradientBoostingClassifier]:
    # TODO: Write the necessary code to train a gradient boosting binary classification model and calculate accuracy, recall, precision, fscore, 
    # false positive rate, false negative rate and area under the reciever operator curve given a train and test dataframe and train and test 
    # target series and keyword arguments for the gradient boosting model

    model = GradientBoostingClassifier()
    train_metrics = {
        "accuracy" : 0,
        "recall" : 0,
        "precision" : 0,
        "fscore" : 0,
        "fpr" : 0,
        "fnr" : 0,
        "roc_auc" : 0
        }
    test_metrics = {
        "accuracy" : 0,
        "recall" : 0,
        "precision" : 0,
        "fscore" : 0,
        "fpr" : 0,
        "fnr" : 0,
        "roc_auc" : 0
        }

    gb_importance = pd.DataFrame()
    gb_metrics = ModelMetrics("Gradient Boosting",train_metrics,test_metrics,gb_importance)

    return gb_metrics,model

In [ ]:
# Run this cell to test your code
gb_kwargs = {'n_estimators':150,'criterion':'squared_error','max_depth':3,'random_state':0}
# Answers for Wine Dataset
with open(os.path.join(os.getcwd(),"..","task4","pkl_files","gb_metrics.pkl"), 'rb') as file:
    gb_metrics_ans = pickle.load(file)
train_accuracy_ans = gb_metrics_ans.train_metrics["accuracy"]
train_recall_ans = gb_metrics_ans.train_metrics["recall"]
train_precision_ans = gb_metrics_ans.train_metrics["precision"]
train_fscore_ans = gb_metrics_ans.train_metrics["fscore"]
train_fpr_ans = gb_metrics_ans.train_metrics["fpr"]
train_fnr_ans = gb_metrics_ans.train_metrics["fnr"]
train_roc_auc_ans = gb_metrics_ans.train_metrics["roc_auc"]

test_accuracy_ans = gb_metrics_ans.test_metrics["accuracy"]
test_recall_ans = gb_metrics_ans.test_metrics["recall"]
test_precision_ans = gb_metrics_ans.test_metrics["precision"]
test_fscore_ans = gb_metrics_ans.test_metrics["fscore"]
test_fpr_ans = gb_metrics_ans.test_metrics["fpr"]
test_fnr_ans = gb_metrics_ans.test_metrics["fnr"]
test_roc_auc_ans = gb_metrics_ans.test_metrics["roc_auc"]

# Calculate Metrics with Student's Function
gb_metrics,_ = calculate_gradient_boosting_metrics(train_features,test_features,train_targets,test_targets,gb_kwargs)

train_accuracy = gb_metrics.train_metrics["accuracy"]
assert train_accuracy == train_accuracy_ans
train_recall = gb_metrics.train_metrics["recall"]
assert train_recall == train_recall_ans
train_precision = gb_metrics.train_metrics["precision"]
assert train_precision == train_precision_ans
train_fscore = gb_metrics.train_metrics["fscore"]
assert train_fscore == train_fscore_ans
train_fpr = gb_metrics.train_metrics["fpr"]
assert train_fpr == train_fpr_ans
train_fnr = gb_metrics.train_metrics["fnr"]
assert train_fnr == train_fnr_ans
train_roc_auc = gb_metrics.train_metrics["roc_auc"]
assert train_roc_auc == train_roc_auc_ans
test_accuracy = gb_metrics.test_metrics["accuracy"]
assert test_accuracy == test_accuracy_ans
test_recall = gb_metrics.test_metrics["recall"]
assert test_recall == test_recall_ans
test_precision = gb_metrics.test_metrics["precision"]
assert test_precision == test_precision_ans
test_fscore = gb_metrics.test_metrics["fscore"]
assert test_fscore == test_fscore_ans
test_fpr = gb_metrics.test_metrics["fpr"]
assert test_fpr == test_fpr_ans
test_fnr = gb_metrics.test_metrics["fnr"]
assert test_fnr == test_fnr_ans
test_roc_auc = gb_metrics.test_metrics["roc_auc"]
print(test_roc_auc)
assert test_roc_auc == test_roc_auc_ans
importance_df = gb_metrics.feat_imp_df
if compare_submission_to_answer_df(importance_df.round(4),gb_importance_df_ans,"GB Feature Importance DF"):
    print('Passed...')
else:
    print('Failed...')

# You have successfully reached the end of this notebook.